## Задание №1

В программе ниже сформирована обучающая x_train, y_train и тестовая x_test, y_test выборки для задачи бинарной классификации с метками классов {-1, +1}.

Вычислите композицию T = 10 моделей по алгоритму AdaBoost. В качестве алгоритмов $b_1(x), ..., b_T(x)$ используйте решающие деревья с максимальной глубиной max_depth = 3 и критерием Джини:

`b_t = DecisionTreeClassifier(criterion='gini', max_depth=max_depth)`

Каждое дерево следует обучать на выборке x_train, y_train с весами $w_1, ..., w_l$ объектов выборки x_train:

`b_t.fit(x_train, y_train, sample_weight=w)`

где w - массив NumPy с весами соответствующих объектов выборки x_train.

Начальные значения весов следует выбрать равными:

`w = np.ones(len(x_train)) / len(x_train)`    # начальные значения весов для объектов выборки

А, затем, для каждого значения t = 1, ... ,T вычислять множитель $α_t$:
$$N(b_t) = \sum_ {i=1}^l w_i * [b_t(x_i) \not = y_i] $$

$$α_t = \frac{1}{2} ln \frac{1-N(b_t)}{N(b_t)} $$

и пересчитывать веса $w_1, ..., w_l$:

$w_i = w_i * exp(-α_ty_ib_t(x_i)), i = 1, ..., l$


с их последующей перенормировкой:

$$w_i = w_i/\sum_ {i=1}^l w_j, i=1,...,l$$

На основе полученных моделей $b_1(x), ..., b_T(x)$ и множителей $α_1, ..., α_T$ прогноз строится по формуле:

$$a(x) = sign(\sum_ {t=1}^T α_t * b_t(x)) $$

Используя эту формулу, выполните классификацию образов тестовой выборки x_test. Результат классификации сохраните в списке predict. 

Вычислите показатель качества для тестовой выборки:

$$Q(a,X) = \sum_ {i=1}^l [a(x_i) \not = y_i] $$

и результат сохраните в переменной ``Q``.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


np.random.seed(0)
n_feature = 2

# исходные параметры для формирования образов обучающей выборки
r1 = 0.7
D1 = 3.0
mean1 = [3, 7]
V1 = [[D1 * r1 ** abs(i-j) for j in range(n_feature)] for i in range(n_feature)]

r2 = 0.5
D2 = 2.0
mean2 = [4, 2]
V2 = [[D2 * r2 ** abs(i-j) for j in range(n_feature)] for i in range(n_feature)]

# моделирование обучающей выборки
N1, N2 = 1000, 1200
x1 = np.random.multivariate_normal(mean1, V1, N1).T
x2 = np.random.multivariate_normal(mean2, V2, N2).T

data_x = np.hstack([x1, x2]).T
data_y = np.hstack([np.ones(N1) * -1, np.ones(N2)])


x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, random_state=123,test_size=0.3, shuffle=True)
w = np.ones(len(x_train)) / len(x_train)    # начальные значения весов для объектов выборки
max_depth = 3
T = 10
models = []
alphas = []

for t in range(T):
    models.append(DecisionTreeClassifier(criterion='gini', max_depth=max_depth))
    models[t].fit(x_train, y_train, sample_weight=w)
    
    pred = models[t].predict(x_train)
    N = np.sum(w * (pred != y_train))

    alpha = 0.5 * np.log((1-N)/N)
    alphas.append(alpha)

    w *= np.exp(-alpha * y_train * pred)
    w /= np.sum(w)

predict = np.sign(sum((alphas[i] * models[i].predict(x_test) for i in range(len(models)))))

Q = np.sum(predict != y_test)

## Задание №2

В программе ниже сформировано множество точек x, y функции:

$f(x) = 2 * cos(x) + 0.5 * sin(2x) - 0.2 * sin(4x) $

на интервале [-3; 3] с шагом 0.1

Реализуйте алгоритм AdaBoost для аппроксимации функции f(x). 
Для этого используйте композицию из T = 6 моделей $b_1, ..., b_T$, каждая из которых представляет собой решающее дерево с максимальной глубиной max_depth = 3:

`b_t = DecisionTreeRegressor(max_depth=max_depth)`

Выполните последовательное построение (обучение) решающих деревьев на остатках функции, не учтенных предыдущими моделями:

$$S_{i,t} = y_i - \sum_ {j=1}^T b_j(x_i), i = 1, 2, ..., l $$

Выполните восстановление функции f(x) по массиву x с помощью полученной композиции:

$$a(x) = \sum_ {j=1}^T b_j(x) $$

Вычислите показатель качества восстановления:

$$Q_T = \frac {1}{l} \sum_ {i=1}^l (y_i - a(x_i))^2 $$

результат сохраните в переменной ``QT``.

In [3]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


x = np.arange(-3, 3, 0.1).reshape(-1, 1)
y = 2 * np.cos(x) + 0.5 * np.sin(2*x) - 0.2 * np.sin(4*x)

T = 6
max_depth = 3
models = []
s = np.array(y.ravel())

for t in range(T):
    models.append(DecisionTreeRegressor(max_depth=max_depth))
    models[-1].fit(x,s)

    bx = models[-1].predict(x)
    
    s -= models[-1].predict(x)

yy = sum(models[i].predict(x) for i in range(T))


QT = np.mean([(y[i][0] - yy[i]) ** 2 for i in range(len(yy))])